In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import os
import torch

from transformers import GPT2Tokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Assign eos_token (end-of-text) as the padding token
model = GPT2LMHeadModel.from_pretrained("gpt2")



In [3]:
# Load and process the Ramayana text data
ramayana_train_file = '/kaggle/input/ramayana-processed/ramayana train.txt'  # Replace with your dataset file path

with open(ramayana_train_file, 'r', encoding='utf-8') as file:
    ramayana_text = file.read()

validation_file_path = '/kaggle/input/ramayana-processed/validation.txt'  # Replace with your validation dataset file path

In [4]:
with open(validation_file_path, 'r', encoding='utf-8') as file:
    validation_text = file.read()
validation_dir = '/kaggle/working/validation/'  # Replace with your desired directory
os.makedirs(validation_dir, exist_ok=True)



In [5]:
validation_file_path = os.path.join(validation_dir, 'validation_ramayana.txt')
with open(validation_file_path, 'w', encoding='utf-8') as file:
    file.write(validation_text)

In [6]:
from torch.utils.data import Dataset

class RamayanaDataset(Dataset):
    def __init__(self, ramayana_text):
        self.tokenized_text = ramayana_text

    def __len__(self):
        return len(self.tokenized_text)

    def __getitem__(self, idx):
        return {"input_ids": self.tokenized_text[idx], "labels": self.tokenized_text[idx]}



In [7]:
train_dataset = RamayanaDataset(ramayana_text)



In [8]:
training_args = TrainingArguments(
    output_dir="./ramayana_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    prediction_loss_only=True,
    report_to="none",
    eval_steps=500,  # Evaluate every X training steps
    evaluation_strategy="steps")

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_dataset,
    eval_dataset=validation_text
)


In [ ]:
trainer.train()

In [12]:
filename = '/kaggle/input/samskrta/samskrta.h5'

In [13]:
import pickle
model = pickle.load(open(filename, 'rb'))


In [40]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

prompt = "नारदं परिपप्रच्छ वाल्मीकि"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
max_length = 100 # Adjust the length of the generated text as needed
generated_text_ids = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

# Decode the generated text
generated_text = tokenizer.decode(generated_text_ids[0], skip_special_tokens=True)

# Print the generated text
print("Generated Text:")
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
नारदं परिपप्रच्छ वाल्मीकि महता यथा

गुणैः शोभयेऽसृजूषटौघशङ
